In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [2]:
cv=LeaveOneOut()

In [3]:
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("label.csv",header=None)
y=np.array(y)
#print(X.shape)

In [4]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    param1=["best","random"]
    param_grid={"splitter":param1}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["splitter"]
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
    param_grid={"max_depth":param2}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["max_depth"]
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(10,i,50))  ## 设置max_features范围
    param_grid={"max_features":param3}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
#     print(datetime.datetime.now())
#     print("best parameters:{}".format(grid_search.best_params_))                        #输出最优网格化参数
#     print("best cross-validation score:{:.3f}".format(grid_search.best_score_))         #成员提供优化过程中观察到的最好评分
#     print("best estimator:\n{}".format(grid_search.best_estimator_))
    Predict_Y=[]    #存放预测值
    Prob_Y=[]       #存放预测概率
    Really_Y=y  #存放真实标签
    for train_index,test_index in cv.split(X):
        estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=grid_search.best_params_["max_features"])
        estimator.fit(X[train_index],y[train_index])
        Predict_Y.append(estimator.predict(X[test_index])[0])
        Prob_Y.append(estimator.predict_proba(X[test_index]))
    Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
    Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
    Prob_Y=pd.DataFrame(Prob_Y)
    Prob_Y.columns=["0","1"]
    confusion_matrix(Really_Y,Predict_Y)
    TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
    FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
    FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
    TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
    ACC=accuracy_score(Really_Y,Predict_Y)
    F1_score=f1_score(Really_Y,Predict_Y)
    AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
    Recall=recall_score(Really_Y,Predict_Y)
    Precision=precision_score(Really_Y,Predict_Y)
    S=p1
    D=p2
    M=grid_search.best_params_["max_features"]
#     print("ACC:",ACC)
#     print("f1_score:",F1_score)
#     print("AUC:",AUC)
    return TN,FN,FP,TP,ACC,Recall,Precision,F1_score,AUC,S,D,M

In [7]:
file_path = 'E:\datatest\PseDPC\d3\d31'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
311.csv
**************************************************
2022-02-24 15:06:54.224541
2022-02-24 15:06:56.296024
2022-02-24 15:07:06.943512
##################################################
312.csv
**************************************************
2022-02-24 15:07:09.807849
2022-02-24 15:07:11.774608
2022-02-24 15:07:21.717984
##################################################
313.csv
**************************************************
2022-02-24 15:07:24.385845
2022-02-24 15:07:26.786423
2022-02-24 15:07:38.800307
##################################################
314.csv
**************************************************
2022-02-24 15:07:41.526984
2022-02-24 15:07:43.385048
2022-02-24 15:07:52.451755
##################################################
315.csv
**************************************************
2022-02-24 15:07:55.353991
2022-02-24 15:07:57.332696
2022-02-24 15:08:06.901096
#############################################

In [8]:
file_path = 'E:\datatest\PseDPC\d3\d32'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
321.csv
**************************************************
2022-02-24 15:09:13.944841
2022-02-24 15:09:15.905595
2022-02-24 15:09:25.899308
##################################################
322.csv
**************************************************
2022-02-24 15:09:29.068828
2022-02-24 15:09:33.650572
2022-02-24 15:09:47.463919
##################################################
323.csv
**************************************************
2022-02-24 15:09:51.887084
2022-02-24 15:09:55.756188
2022-02-24 15:09:59.752496
##################################################
324.csv
**************************************************
2022-02-24 15:10:01.491842
2022-02-24 15:10:04.349232
2022-02-24 15:10:15.847469
##################################################
325.csv
**************************************************
2022-02-24 15:10:18.477432
2022-02-24 15:10:20.822159
2022-02-24 15:10:32.122973
#############################################

In [9]:
file_path = 'E:\datatest\PseDPC\d3\d33'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
331.csv
**************************************************
2022-02-24 15:11:39.257227
2022-02-24 15:11:41.353645
2022-02-24 15:11:50.558012
##################################################
332.csv
**************************************************
2022-02-24 15:11:54.582224
2022-02-24 15:11:57.077548
2022-02-24 15:12:07.921535
##################################################
333.csv
**************************************************
2022-02-24 15:12:11.031216
2022-02-24 15:12:12.931132
2022-02-24 15:12:22.800747
##################################################
334.csv
**************************************************
2022-02-24 15:12:26.101900
2022-02-24 15:12:28.795687
2022-02-24 15:12:39.238745
##################################################
335.csv
**************************************************
2022-02-24 15:12:42.410260
2022-02-24 15:12:44.510641
2022-02-24 15:12:48.051168
#############################################

In [10]:
file_path = 'E:\datatest\PseDPC\d3\d34'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
341.csv
**************************************************
2022-02-24 15:13:55.042401
2022-02-24 15:13:57.224564
2022-02-24 15:14:07.155999
##################################################
342.csv
**************************************************
2022-02-24 15:14:09.812882
2022-02-24 15:14:11.910303
2022-02-24 15:14:21.382955
##################################################
343.csv
**************************************************
2022-02-24 15:14:24.007904
2022-02-24 15:14:26.193058
2022-02-24 15:14:37.418061
##################################################
344.csv
**************************************************
2022-02-24 15:14:40.395060
2022-02-24 15:14:42.842512
2022-02-24 15:14:54.698790
##################################################
345.csv
**************************************************
2022-02-24 15:14:58.001954
2022-02-24 15:15:00.207053
2022-02-24 15:15:11.421050
#############################################

In [11]:
file_path = 'E:\datatest\PseDPC\d3\d35'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
351.csv
**************************************************
2022-02-24 15:16:34.763185
2022-02-24 15:16:37.202637
2022-02-24 15:16:48.520378
##################################################
352.csv
**************************************************
2022-02-24 15:16:51.919262
2022-02-24 15:16:54.303883
2022-02-24 15:17:05.990403
##################################################
353.csv
**************************************************
2022-02-24 15:17:09.126014
2022-02-24 15:17:11.632833
2022-02-24 15:17:23.496092
##################################################
354.csv
**************************************************
2022-02-24 15:17:28.079409
2022-02-24 15:17:30.647537
2022-02-24 15:17:43.105228
##################################################
355.csv
**************************************************
2022-02-24 15:17:46.130113
2022-02-24 15:17:48.297315
2022-02-24 15:18:01.773084
#############################################

In [12]:
file_path = 'E:\datatest\PseDPC\d3\d36'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
361.csv
**************************************************
2022-02-24 15:19:15.263462
2022-02-24 15:19:17.446643
2022-02-24 15:19:21.146754
##################################################
362.csv
**************************************************
2022-02-24 15:19:22.879086
2022-02-24 15:19:25.530990
2022-02-24 15:19:29.398642
##################################################
363.csv
**************************************************
2022-02-24 15:19:31.067178
2022-02-24 15:19:33.192507
2022-02-24 15:19:44.690728
##################################################
364.csv
**************************************************
2022-02-24 15:19:48.069710
2022-02-24 15:19:50.451317
2022-02-24 15:20:02.288191
##################################################
365.csv
**************************************************
2022-02-24 15:20:05.282152
2022-02-24 15:20:07.845190
2022-02-24 15:20:12.024669
#############################################

In [13]:
file_path = 'E:\datatest\PseDPC\d3\d37'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
371.csv
**************************************************
2022-02-24 15:21:15.316421
2022-02-24 15:21:17.471654
2022-02-24 15:21:28.216434
##################################################
372.csv
**************************************************
2022-02-24 15:21:31.637303
2022-02-24 15:21:34.133603
2022-02-24 15:21:46.267161
##################################################
373.csv
**************************************************
2022-02-24 15:21:49.091733
2022-02-24 15:21:51.598875
2022-02-24 15:22:03.531580
##################################################
374.csv
**************************************************
2022-02-24 15:22:06.510610
2022-02-24 15:22:08.957064
2022-02-24 15:22:20.685684
##################################################
375.csv
**************************************************
2022-02-24 15:22:24.350003
2022-02-24 15:22:26.897087
2022-02-24 15:22:39.172430
#############################################

In [14]:
file_path = 'E:\datatest\PseDPC\d3\d38'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
381.csv
**************************************************
2022-02-24 15:23:53.540480
2022-02-24 15:23:55.731596
2022-02-24 15:24:06.959660
##################################################
382.csv
**************************************************
2022-02-24 15:24:10.211937
2022-02-24 15:24:12.343235
2022-02-24 15:24:24.056895
##################################################
383.csv
**************************************************
2022-02-24 15:24:26.961124
2022-02-24 15:24:29.034577
2022-02-24 15:24:39.954362
##################################################
384.csv
**************************************************
2022-02-24 15:24:42.717988
2022-02-24 15:24:44.816353
2022-02-24 15:24:55.877132
##################################################
385.csv
**************************************************
2022-02-24 15:24:58.909052
2022-02-24 15:25:01.126089
2022-02-24 15:25:12.573461
#############################################

In [15]:
file_path = 'E:\datatest\PseDPC\d3\d39'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
391.csv
**************************************************
2022-02-24 15:26:26.498674
2022-02-24 15:26:29.088745
2022-02-24 15:26:41.033786
##################################################
392.csv
**************************************************
2022-02-24 15:26:44.660084
2022-02-24 15:26:47.150423
2022-02-24 15:26:59.747787
##################################################
393.csv
**************************************************
2022-02-24 15:27:03.188539
2022-02-24 15:27:05.456464
2022-02-24 15:27:17.242908
##################################################
394.csv
**************************************************
2022-02-24 15:27:19.968616
2022-02-24 15:27:22.523780
2022-02-24 15:27:26.982850
##################################################
395.csv
**************************************************
2022-02-24 15:27:28.941612
2022-02-24 15:27:31.468847
2022-02-24 15:27:43.675221
#############################################

In [16]:
file_path = 'E:\datatest\PseDPC\d3\d310'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
3101.csv
**************************************************
2022-02-24 15:28:41.840568
2022-02-24 15:28:44.205241
2022-02-24 15:28:56.713775
##################################################
3102.csv
**************************************************
2022-02-24 15:28:59.767604
2022-02-24 15:29:02.090411
2022-02-24 15:29:13.071011
##################################################
3103.csv
**************************************************
2022-02-24 15:29:15.878499
2022-02-24 15:29:18.177351
2022-02-24 15:29:30.092711
##################################################
3104.csv
**************************************************
2022-02-24 15:29:34.044137
2022-02-24 15:29:36.483610
2022-02-24 15:29:40.460991
##################################################
3105.csv
**************************************************
2022-02-24 15:29:41.546065
2022-02-24 15:29:43.779092
2022-02-24 15:29:47.649736
########################################

In [17]:
file_path = 'E:\datatest\PseDPC\d3\d311'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
3111.csv
**************************************************
2022-02-24 15:30:48.524869
2022-02-24 15:30:50.597323
2022-02-24 15:31:00.987547
##################################################
3112.csv
**************************************************
2022-02-24 15:31:04.020442
2022-02-24 15:31:06.429991
2022-02-24 15:31:09.742101
##################################################
3113.csv
**************************************************
2022-02-24 15:31:10.715497
2022-02-24 15:31:13.123056
2022-02-24 15:31:16.749354
##################################################
3114.csv
**************************************************
2022-02-24 15:31:17.815512
2022-02-24 15:31:20.107404
2022-02-24 15:31:24.722528
##################################################
3115.csv
**************************************************
2022-02-24 15:31:25.901342
2022-02-24 15:31:28.513381
2022-02-24 15:31:32.006009
########################################

In [18]:
file_path = 'E:\datatest\PseDPC\d3\d312'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
3121.csv
**************************************************
2022-02-24 15:32:33.080315
2022-02-24 15:32:35.269481
2022-02-24 15:32:46.871439
##################################################
3122.csv
**************************************************
2022-02-24 15:32:49.734756
2022-02-24 15:32:52.346767
2022-02-24 15:33:04.962775
##################################################
3123.csv
**************************************************
2022-02-24 15:33:08.555186
2022-02-24 15:33:11.247959
2022-02-24 15:33:23.646787
##################################################
3124.csv
**************************************************
2022-02-24 15:33:27.291035
2022-02-24 15:33:29.939970
2022-02-24 15:33:42.698813
##################################################
3125.csv
**************************************************
2022-02-24 15:33:46.120657
2022-02-24 15:33:48.557139
2022-02-24 15:33:52.422817
########################################

In [19]:
file_path = 'E:\datatest\PseDPC\d3\d313'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
3131.csv
**************************************************
2022-02-24 15:34:24.810143
2022-02-24 15:34:27.090044
2022-02-24 15:34:39.263524
##################################################
3132.csv
**************************************************
2022-02-24 15:34:41.978233
2022-02-24 15:34:44.598223
2022-02-24 15:34:56.926240
##################################################
3133.csv
**************************************************
2022-02-24 15:34:59.724752
2022-02-24 15:35:02.143281
2022-02-24 15:35:05.814459
##################################################
3134.csv
**************************************************
2022-02-24 15:35:06.989316
2022-02-24 15:35:09.550485
2022-02-24 15:35:21.831606
##################################################
3135.csv
**************************************************
2022-02-24 15:35:24.729851
2022-02-24 15:35:27.209246
2022-02-24 15:35:38.962772
########################################

In [20]:
file_path = 'E:\datatest\PseDPC\d3\d314'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
3141.csv
**************************************************
2022-02-24 15:36:52.414252
2022-02-24 15:36:55.022274
2022-02-24 15:37:07.066072
##################################################
3142.csv
**************************************************
2022-02-24 15:37:11.013490
2022-02-24 15:37:13.505822
2022-02-24 15:37:26.082205
##################################################
3143.csv
**************************************************
2022-02-24 15:37:29.638662
2022-02-24 15:37:32.813511
2022-02-24 15:37:46.872962
##################################################
3144.csv
**************************************************
2022-02-24 15:37:49.986633
2022-02-24 15:37:52.708350
2022-02-24 15:38:06.807628
##################################################
3145.csv
**************************************************
2022-02-24 15:38:10.246461
2022-02-24 15:38:12.926291
2022-02-24 15:38:26.087048
########################################

In [21]:
file_path = 'E:\datatest\PseDPC\d3\d315'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
3151.csv
**************************************************
2022-02-24 15:39:43.734330
2022-02-24 15:39:46.049141
2022-02-24 15:39:58.570634
##################################################
3152.csv
**************************************************
2022-02-24 15:40:02.053322
2022-02-24 15:40:04.336208
2022-02-24 15:40:15.683820
##################################################
3153.csv
**************************************************
2022-02-24 15:40:18.762582
2022-02-24 15:40:21.166153
2022-02-24 15:40:32.261494
##################################################
3154.csv
**************************************************
2022-02-24 15:40:35.739195
2022-02-24 15:40:38.148716
2022-02-24 15:40:49.822482
##################################################
3155.csv
**************************************************
2022-02-24 15:40:52.512285
2022-02-24 15:40:54.784234
2022-02-24 15:41:06.099932
########################################

In [22]:
file_path = 'E:\datatest\PseDPC\d3\d316'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
3161.csv
**************************************************
2022-02-24 15:42:41.225024
2022-02-24 15:42:45.527073
2022-02-24 15:42:57.789235
##################################################
3162.csv
**************************************************
2022-02-24 15:43:08.978333
2022-02-24 15:43:12.397152
2022-02-24 15:43:23.650052
##################################################
3163.csv
**************************************************
2022-02-24 15:43:33.537631
2022-02-24 15:43:36.664225
2022-02-24 15:43:47.620911
##################################################
3164.csv
**************************************************
2022-02-24 15:43:57.344894
2022-02-24 15:44:00.340879
2022-02-24 15:44:08.244732
##################################################
3165.csv
**************************************************
2022-02-24 15:44:17.767254
2022-02-24 15:44:20.834049
2022-02-24 15:44:29.665421
########################################

In [23]:
file_path = 'E:\datatest\PseDPC\d3\d317'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
3171.csv
**************************************************
2022-02-24 15:46:18.797439
2022-02-24 15:46:22.267183
2022-02-24 15:46:31.248128
##################################################
3172.csv
**************************************************
2022-02-24 15:46:41.884669
2022-02-24 15:46:45.559837
2022-02-24 15:46:54.417139
##################################################
3173.csv
**************************************************
2022-02-24 15:47:02.080635
2022-02-24 15:47:05.997157
2022-02-24 15:47:17.871387
##################################################
3174.csv
**************************************************
2022-02-24 15:47:28.732899
2022-02-24 15:47:31.922366
2022-02-24 15:47:40.356867
##################################################
3175.csv
**************************************************
2022-02-24 15:47:48.227778
2022-02-24 15:47:51.300590
2022-02-24 15:48:03.066078
########################################

In [24]:
file_path = 'E:\datatest\PseDPC\d3\d318'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
3181.csv
**************************************************
2022-02-24 15:49:52.145238
2022-02-24 15:49:56.187423
2022-02-24 15:50:04.473254
##################################################
3182.csv
**************************************************
2022-02-24 15:50:15.962514
2022-02-24 15:50:19.453176
2022-02-24 15:50:28.180825
##################################################
3183.csv
**************************************************
2022-02-24 15:50:38.775500
2022-02-24 15:50:42.569362
2022-02-24 15:50:50.716558
##################################################
3184.csv
**************************************************
2022-02-24 15:50:58.711174
2022-02-24 15:51:01.825829
2022-02-24 15:51:12.773518
##################################################
3185.csv
**************************************************
2022-02-24 15:51:25.405750
2022-02-24 15:51:28.767753
2022-02-24 15:51:40.768617
########################################

In [25]:
file_path = 'E:\datatest\PseDPC\d3\d319'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
3191.csv
**************************************************
2022-02-24 15:53:20.266410
2022-02-24 15:53:23.741143
2022-02-24 15:53:35.648285
##################################################
3192.csv
**************************************************
2022-02-24 15:53:49.092315
2022-02-24 15:53:52.726564
2022-02-24 15:54:02.132400
##################################################
3193.csv
**************************************************
2022-02-24 15:54:12.275299
2022-02-24 15:54:16.074097
2022-02-24 15:54:28.309362
##################################################
3194.csv
**************************************************
2022-02-24 15:54:41.231789
2022-02-24 15:54:44.342466
2022-02-24 15:54:55.133622
##################################################
3195.csv
**************************************************
2022-02-24 15:55:06.476275
2022-02-24 15:55:09.532071
2022-02-24 15:55:20.185568
########################################

In [26]:
file_path = 'E:\datatest\PseDPC\d3\d320'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
3201.csv
**************************************************
2022-02-24 15:57:11.154438
2022-02-24 15:57:14.684964
2022-02-24 15:57:23.124385
##################################################
3202.csv
**************************************************
2022-02-24 15:57:30.086779
2022-02-24 15:57:33.972361
2022-02-24 15:57:43.681411
##################################################
3203.csv
**************************************************
2022-02-24 15:57:54.293023
2022-02-24 15:57:57.706892
2022-02-24 15:58:11.209282
##################################################
3204.csv
**************************************************
2022-02-24 15:58:24.396965
2022-02-24 15:58:28.037226
2022-02-24 15:58:40.493897
##################################################
3205.csv
**************************************************
2022-02-24 15:58:52.859813
2022-02-24 15:58:56.152005
2022-02-24 15:59:08.155910
########################################